In [14]:
import torch 
import matplotlib.pyplot as plt 
import numpy as np


In [15]:
orig_video_length = 447
frame_list = np.ones((447, 1242, 3))


inference_length = 32 # Complete length of processed Context
keyframe_list = [44, 12] # Distance to the last frame of the processed context. Exception 0 means always frame 0  START WITH HIGHEST DISTANCE, WE ALWAYS USE KEYFRAME 0
align_on_each_new_frame = True
max_keyframe = max(keyframe_list)

assert inference_length > len(keyframe_list) + 2 # Because 0 is always a keyframe; 1 new Frame must be predicted 

# In Loop
cur_frame = np.ones((1, 1, 3, 280, 924))
layer_1, layer_2, layer_3, layer_4 = torch.ones((1, 48, 80, 264)), torch.ones((1, 96, 40, 132)), torch.ones((1, 192, 20, 66)), torch.ones((1, 384, 10, 33))

motion_features = (np.ones((inference_length, 48, 80, 264)), np.ones((inference_length, 96, 40, 132)), np.ones((inference_length, 192, 20, 66)), 
                   np.ones((inference_length, 384, 10, 33)))

In [22]:
# prepare tensor for saving: 
features_l1 = torch.zeros((inference_length+max_keyframe, *layer_1.size()))
features_l2 = torch.zeros((inference_length+max_keyframe, *layer_2.size()))
features_l3 = torch.zeros((inference_length+max_keyframe, *layer_3.size()))
features_l4 = torch.zeros((inference_length+max_keyframe, *layer_4.size()))

# We need the following lists: 
# Motion_feature_idx: Which motion features to keep -- Should be always the same 




# Depth_pred_idx: which depth idx to predict on the prepared motion feautes 
# Aling depth idx: On which predicted idx to aling 


distance_to_batch = [keyframe_list[idx] + (inference_length-len(keyframe_list)) for idx in range(len(keyframe_list))]

static_keyframes = []
for idx in range(len(keyframe_list)):
    if inference_length > keyframe_list[idx]:
        static_keyframes.append(inference_length - keyframe_list[idx])
    elif inference_length <= keyframe_list[idx]:
        static_keyframes.append(idx+1)
assert not len(static_keyframes) != len(set(static_keyframes)), f'Setup leads to duplicates in the keyframes: {static_keyframes}'



Abs_depth_idx = [] #Absoluter index des Frames 
Align_idx = [] #Relativer index des frames within prediction batch 
use_feature_idx = [] #index der zu benutzenden feauters 
for frame_idx in range(inference_length-1, inference_length+max_keyframe+1, 1):

    tmp_batch_idx = [idx for idx in range(frame_idx-(inference_length-1), frame_idx, 1)]
    tmp_batch_idx[0] = 0 # We always use first frame as reference
    align_idx_batch = [0]
    
    for idx, static_keyframe in enumerate(static_keyframes):
        if static_keyframe in tmp_batch_idx:
            align_idx_batch.append(tmp_batch_idx.index(static_keyframe))
        else:
            if frame_idx-distance_to_batch[idx] <= static_keyframe:
                tmp_batch_idx[idx+1] = static_keyframe
                align_idx_batch.append(idx+1)
            else:
                tmp_batch_idx[idx+1] = static_keyframe + (frame_idx - distance_to_batch[idx] - static_keyframe)
                align_idx_batch.append(idx+1)
    use_feature_idx.append(tmp_batch_idx)
    Align_idx.append(align_idx_batch)
# up to inference_length+max_keyframe use_featuer_idx[:len(keyframe_list)+1] == Align_idx == Depth_pred_idx, da alles was bisher predicted wurde drin auch noch gespeichert ist. 
# Danach schiebt sich Align_idx und Depth_pred_idx immer um die differenz von frame_idx - inference_length+max_keyframe weiter. 
for i in range(len(use_feature_idx)):
    print(use_feature_idx[i])
    print(Align_idx[i])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 20]
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 19]
[0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[0, 1, 18]
[0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
[0, 1, 17]
[0, 1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
[0, 1, 16]
[0, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
[0, 1, 15]
[0, 1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
[0, 1, 14]
[0, 1, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,

In [9]:
static_keyframes


[1, 20]

In [7]:
distance_to_batch

[74, 42]

In [31]:
first_frame_depth_idx

NameError: name 'first_frame_depth_idx' is not defined

In [32]:
static_keyframes

[1, 2]